In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [33]:
df = pd.read_csv("yt_comments.csv")

In [34]:

filtered_taylor_swift = df[df['Name'] == 'Taylor Swift']

In [35]:
filtered_mrbeast = df[df['Name'] == 'MrBeast']


In [38]:
example = filtered_taylor_swift['comment'][10]
lower_case = example.lower()
cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))    

In [42]:
tockenized_text = word_tokenize(cleaned_text, "english")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/harshsahu/nltk_data'
    - '/Users/harshsahu/anaconda3/envs/data_science/nltk_data'
    - '/Users/harshsahu/anaconda3/envs/data_science/share/nltk_data'
    - '/Users/harshsahu/anaconda3/envs/data_science/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
